# TrustCall

https://github.com/hinthornw/trustcall

In [2]:
# 複雑なスキーマ
from typing import List, Optional

from pydantic import BaseModel


class OutputFormat(BaseModel):
    preference: str
    sentence_preference_revealed: str


class TelegramPreferences(BaseModel):
    preferred_encoding: Optional[List[OutputFormat]] = None
    favorite_telegram_operators: Optional[List[OutputFormat]] = None
    preferred_telegram_paper: Optional[List[OutputFormat]] = None


class MorseCode(BaseModel):
    preferred_key_type: Optional[List[OutputFormat]] = None
    favorite_morse_abbreviations: Optional[List[OutputFormat]] = None


class Semaphore(BaseModel):
    preferred_flag_color: Optional[List[OutputFormat]] = None
    semaphore_skill_level: Optional[List[OutputFormat]] = None


class TrustFallPreferences(BaseModel):
    preferred_fall_height: Optional[List[OutputFormat]] = None
    trust_level: Optional[List[OutputFormat]] = None
    preferred_catching_technique: Optional[List[OutputFormat]] = None


class CommunicationPreferences(BaseModel):
    telegram: TelegramPreferences
    morse_code: MorseCode
    semaphore: Semaphore


class UserPreferences(BaseModel):
    communication_preferences: CommunicationPreferences
    trust_fall_preferences: TrustFallPreferences


class TelegramAndTrustFallPreferences(BaseModel):
    pertinent_user_preferences: UserPreferences

In [5]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")
bound = llm.with_structured_output(TelegramAndTrustFallPreferences)

conversation = """Operator: How may I assist with your telegram, sir?
Customer: I need to send a message about our trust fall exercise.
Operator: Certainly. Morse code or standard encoding?
Customer: Morse, please. I love using a straight key.
Operator: Excellent. What's your message?
Customer: Tell him I'm ready for a higher fall, and I prefer the diamond formation for catching.
Operator: Done. Shall I use our "Daredevil" paper for this daring message?
Customer: Perfect! Send it by your fastest carrier pigeon.
Operator: It'll be there within the hour, sir."""

bound.invoke(f"""Extract the preferences from the following conversation:
<convo>
{conversation}
</convo>""")

TelegramAndTrustFallPreferences(pertinent_user_preferences=UserPreferences(communication_preferences=CommunicationPreferences(telegram=TelegramPreferences(preferred_encoding=[OutputFormat(preference='Morse code', sentence_preference_revealed='I need to send a message about our trust fall exercise.')], favorite_telegram_operators=[OutputFormat(preference='Daredevil paper', sentence_preference_revealed="Shall I use our 'Daredevil' paper for this daring message?")], preferred_telegram_paper=[OutputFormat(preference='Daredevil paper', sentence_preference_revealed='Perfect! Send it by your fastest carrier pigeon.')]), morse_code=MorseCode(preferred_key_type=[OutputFormat(preference='straight key', sentence_preference_revealed='I love using a straight key.')], favorite_morse_abbreviations=None), semaphore=Semaphore(preferred_flag_color=None, semaphore_skill_level=None)), trust_fall_preferences=TrustFallPreferences(preferred_fall_height=[OutputFormat(preference='higher fall', sentence_prefere

In [6]:
bound = llm.bind_tools([TelegramAndTrustFallPreferences],
                       strict=True,
                       response_format=TelegramAndTrustFallPreferences)

bound.invoke(f"""Extract the preferences from the following conversation:
<convo>
{conversation}
</convo>""")

BadRequestError: Error code: 400 - {'error': {'message': "Invalid schema for function 'TelegramAndTrustFallPreferences': In context=('properties', 'pertinent_user_preferences', 'properties', 'communication_preferences', 'properties', 'telegram', 'properties', 'preferred_encoding'), 'default' is not permitted.", 'type': 'invalid_request_error', 'param': 'tools[0].function.parameters', 'code': 'invalid_function_parameters'}}

In [ ]:
from trustcall import create_extractor

bound = create_extractor(
    llm,
    tools=[TelegramAndTrustFallPreferences],
    tool_choice="TelegramAndTrustFallPreferences",
)

result = bound.invoke(
    f"""Extract the preferences from the following conversation:
<convo>
{conversation}
</convo>"""
)
result["responses"][0]

TelegramAndTrustFallPreferences(pertinent_user_preferences=UserPreferences(communication_preferences=CommunicationPreferences(telegram=TelegramPreferences(preferred_encoding=[OutputFormat(preference='Morse code', sentence_preference_revealed='Morse code')], favorite_telegram_operators=None, preferred_telegram_paper=None), morse_code=MorseCode(preferred_key_type=[OutputFormat(preference='straight key', sentence_preference_revealed='I love using a straight key')], favorite_morse_abbreviations=None), semaphore=Semaphore(preferred_flag_color=None, semaphore_skill_level=None)), trust_fall_preferences=TrustFallPreferences(preferred_fall_height=[OutputFormat(preference='higher', sentence_preference_revealed="I'm ready for a higher fall")], trust_level=None, preferred_catching_technique=[OutputFormat(preference='diamond formation', sentence_preference_revealed='I prefer the diamond formation for catching')])))

In [11]:
from pprint import pprint
pprint(result["responses"][0])

TelegramAndTrustFallPreferences(pertinent_user_preferences=UserPreferences(communication_preferences=CommunicationPreferences(telegram=TelegramPreferences(preferred_encoding=[OutputFormat(preference='Morse code', sentence_preference_revealed='Morse code')], favorite_telegram_operators=None, preferred_telegram_paper=None), morse_code=MorseCode(preferred_key_type=[OutputFormat(preference='straight key', sentence_preference_revealed='I love using a straight key')], favorite_morse_abbreviations=None), semaphore=Semaphore(preferred_flag_color=None, semaphore_skill_level=None)), trust_fall_preferences=TrustFallPreferences(preferred_fall_height=[OutputFormat(preference='higher', sentence_preference_revealed="I'm ready for a higher fall")], trust_level=None, preferred_catching_technique=[OutputFormat(preference='diamond formation', sentence_preference_revealed='I prefer the diamond formation for catching')])))


In [12]:
from typing import Dict, List, Optional

from pydantic import BaseModel


class Address(BaseModel):
    street: str
    city: str
    country: str
    postal_code: str


class Pet(BaseModel):
    kind: str
    name: Optional[str]
    age: Optional[int]


class Hobby(BaseModel):
    name: str
    skill_level: str
    frequency: str


class FavoriteMedia(BaseModel):
    shows: List[str]
    movies: List[str]
    books: List[str]


class User(BaseModel):
    preferred_name: str
    favorite_media: FavoriteMedia
    favorite_foods: List[str]
    hobbies: List[Hobby]
    age: int
    occupation: str
    address: Address
    favorite_color: Optional[str] = None
    pets: Optional[List[Pet]] = None
    languages: Dict[str, str] = {}

In [14]:
initial_user = User(
    preferred_name="アレックス",
    favorite_media=FavoriteMedia(
        shows=[
            "フレンズ",
            "ゲーム・オブ・スローンズ",
            "ブレイキング・バッド",
            "ジ・オフィス",
            "ストレンジャー・シングス",
        ],
        movies=["ショーシャンクの空に", "インセプション", "ダークナイト"],
        books=["1984年", "アラバマ物語", "グレート・ギャツビー"],
    ),
    favorite_foods=["寿司", "ピザ", "タコス", "アイスクリーム", "パスタ", "カレー"],
    hobbies=[
        Hobby(name="読書", skill_level="上級", frequency="毎日"),
        Hobby(name="ハイキング", skill_level="中級", frequency="毎週"),
        Hobby(name="写真撮影", skill_level="初心者", frequency="毎月"),
        Hobby(name="サイクリング", skill_level="中級", frequency="毎週"),
        Hobby(name="水泳", skill_level="上級", frequency="毎週"),
        Hobby(name="カヌー", skill_level="初心者", frequency="毎月"),
        Hobby(name="セーリング", skill_level="中級", frequency="毎月"),
        Hobby(name="織物", skill_level="初心者", frequency="毎週"),
        Hobby(name="絵画", skill_level="中級", frequency="毎週"),
        Hobby(name="料理", skill_level="上級", frequency="毎日"),
    ],
    age=28,
    occupation="ソフトウェアエンジニア",
    address=Address(
        street="123 テックレーン", city="サンフランシスコ", country="アメリカ", postal_code="94105"
    ),
    favorite_color="青",
    pets=[Pet(kind="猫", name="ルナ", age=3)],
    languages={"英語": "ネイティブ", "スペイン語": "中級", "Python": "上級"},
)

In [16]:
initial_user.model_dump()

{'preferred_name': 'アレックス',
 'favorite_media': {'shows': ['フレンズ',
   'ゲーム・オブ・スローンズ',
   'ブレイキング・バッド',
   'ジ・オフィス',
   'ストレンジャー・シングス'],
  'movies': ['ショーシャンクの空に', 'インセプション', 'ダークナイト'],
  'books': ['1984年', 'アラバマ物語', 'グレート・ギャツビー']},
 'favorite_foods': ['寿司', 'ピザ', 'タコス', 'アイスクリーム', 'パスタ', 'カレー'],
 'hobbies': [{'name': '読書', 'skill_level': '上級', 'frequency': '毎日'},
  {'name': 'ハイキング', 'skill_level': '中級', 'frequency': '毎週'},
  {'name': '写真撮影', 'skill_level': '初心者', 'frequency': '毎月'},
  {'name': 'サイクリング', 'skill_level': '中級', 'frequency': '毎週'},
  {'name': '水泳', 'skill_level': '上級', 'frequency': '毎週'},
  {'name': 'カヌー', 'skill_level': '初心者', 'frequency': '毎月'},
  {'name': 'セーリング', 'skill_level': '中級', 'frequency': '毎月'},
  {'name': '織物', 'skill_level': '初心者', 'frequency': '毎週'},
  {'name': '絵画', 'skill_level': '中級', 'frequency': '毎週'},
  {'name': '料理', 'skill_level': '上級', 'frequency': '毎日'}],
 'age': 28,
 'occupation': 'ソフトウェアエンジニア',
 'address': {'street': '123 テックレーン',
  'city': 'サンフランシ

In [15]:
conversation = """友人: やあアレックス、新しい仕事はどう？最近キャリアチェンジしたって聞いたよ。
アレックス: とても順調だよ！新しいデータサイエンティストの役職が気に入ってる。仕事は難しいけどワクワクするよ。オフィスに近いニューヨークの新しいアパートに引っ越したんだ。
友人: それは大きな変化だね！趣味を続ける時間はまだある？
アレックス: まあ、いくつかは減らさなきゃいけなかったかな。最近はセーリングやカヌーはあまりやってないよ。でも、空き時間には機械学習のプロジェクトに夢中なんだ。結構上達してきたと思う。今は中級くらいかな。
友人: 忙しくしてるみたいだね！ルナはどう？
アレックス: ルナは元気だよ。先週4歳になったんだ。実は新しいペットのマックス（2歳のゴールデンレトリバー）と友達になったんだ。マックスはすごく遊び好きなんだよ。
友人: 今は2匹のペットがいるんだね！それは楽しそうだね。ねえ、今週末に『ストレンジャー・シングス』の新シーズンを見ない？
アレックス: 実はその番組にはちょっと興味がなくなってきたんだ。でも新しいシリーズの『マンダロリアン』にハマってる。一緒にそれを見ようよ！あ、それと最近『パラサイト』を見たんだけど、僕の好きな映画のひとつになったよ。
友人: いいね、それ楽しそうだ。食べ物を持っていこうか？君が寿司好きなの覚えてるよ。
アレックス: 寿司は完璧だね！それかタイ料理でもいいかな。最近それにもハマってるんだ。それと、フランス語を練習してて、今は初心者レベルって感じかな。
友人: 素晴らしいね！君はいつも新しいことを学んでるね。料理はどう？
アレックス: 順調だよ！ほぼ毎日料理してる。結構上達してきたと思うよ。
"""

In [24]:
llm = ChatOpenAI(model="gpt-4o")
bound = llm.with_structured_output(User)
naive_result = bound.invoke(
    f"""以下の会話の情報を元にメモリ（JSONドキュメント）を更新してください:
<user_info>
{initial_user.model_dump()}   
</user_info>
<convo>
{conversation}
</convo>"""
)

print("Naive approach result:")
naive_output = naive_result.model_dump()
print(naive_output)

Naive approach result:
{'preferred_name': 'アレックス', 'favorite_media': {'shows': ['フレンズ', 'ゲーム・オブ・スローンズ', 'ブレイキング・バッド', 'ジ・オフィス'], 'movies': ['ショーシャンクの空に', 'インセプション', 'ダークナイト', 'パラサイト'], 'books': ['1984年', 'アラバマ物語', 'グレート・ギャツビー']}, 'favorite_foods': ['寿司', 'ピザ', 'タコス', 'アイスクリーム', 'パスタ', 'カレー', 'タイ料理'], 'hobbies': [{'name': '読書', 'skill_level': '上級', 'frequency': '毎日'}, {'name': 'ハイキング', 'skill_level': '中級', 'frequency': '毎週'}, {'name': '写真撮影', 'skill_level': '初心者', 'frequency': '毎月'}, {'name': 'サイクリング', 'skill_level': '中級', 'frequency': '毎週'}, {'name': '水泳', 'skill_level': '上級', 'frequency': '毎週'}, {'name': '織物', 'skill_level': '初心者', 'frequency': '毎週'}, {'name': '絵画', 'skill_level': '中級', 'frequency': '毎週'}, {'name': '料理', 'skill_level': '上級', 'frequency': '毎日'}, {'name': '機械学習', 'skill_level': '中級', 'frequency': '不明'}], 'age': 28, 'occupation': 'データサイエンティスト', 'address': {'street': '不明', 'city': 'ニューヨーク', 'country': 'アメリカ', 'postal_code': '不明'}, 'favorite_color': '青', 'pets': [{'kind': 

In [25]:
from trustcall import create_extractor

bound = create_extractor(llm, tools=[User])

trustcall_result = bound.invoke(
    {
        "messages": [
            {"role": "user", "content": f"""以下の会話の情報を元にメモリ（JSONドキュメント）を更新してください:
<convo>
{conversation}
</convo>"""}
        ],
        "existing": {"User": initial_user.model_dump()}
    }
)

print("Trustcall approach result:")
trustcall_output = trustcall_result["responses"][0].model_dump()
print(trustcall_output)

Trustcall approach result:
{'preferred_name': 'アレックス', 'favorite_media': {'shows': ['フレンズ', 'ゲーム・オブ・スローンズ', 'ブレイキング・バッド', 'ジ・オフィス', 'マンダロリアン'], 'movies': ['ショーシャンクの空に', 'インセプション', 'ダークナイト', 'パラサイト'], 'books': ['1984年', 'アラバマ物語', 'グレート・ギャツビー']}, 'favorite_foods': ['寿司', 'ピザ', 'タコス', 'アイスクリーム', 'パスタ', 'カレー', 'タイ料理'], 'hobbies': [{'name': '読書', 'skill_level': '上級', 'frequency': '毎日'}, {'name': 'ハイキング', 'skill_level': '中級', 'frequency': '毎週'}, {'name': '写真撮影', 'skill_level': '初心者', 'frequency': '毎月'}, {'name': 'サイクリング', 'skill_level': '中級', 'frequency': '毎週'}, {'name': '水泳', 'skill_level': '上級', 'frequency': '毎週'}, {'name': '織物', 'skill_level': '初心者', 'frequency': '毎週'}, {'name': '絵画', 'skill_level': '中級', 'frequency': '毎週'}, {'name': '料理', 'skill_level': '上級', 'frequency': '毎日'}, {'name': '機械学習プロジェクト', 'skill_level': '中級', 'frequency': '空き時間'}], 'age': 28, 'occupation': 'データサイエンティスト', 'address': {'street': '新しいアパート', 'city': 'ニューヨーク', 'country': 'アメリカ', 'postal_code': '10001'}, 'favorite_